In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Path where testing data is stored
test_data_path = "/content/drive/MyDrive/Dataset/splitset/"

# Get participant IDs from existing training files
test_participants = [
    file.split("_")[0] for file in os.listdir(test_data_path) if file.endswith("_test.mat")
]
test_participants = sorted(set(map(int, test_participants)))  # Ensure unique and sorted IDs

print("Loaded test participants:", test_participants)

Loaded test participants: [4, 5, 7, 9, 13, 20, 25, 26, 27, 28, 34, 38, 40, 41, 44]


In [3]:
# Group trials into 20-second segments for each test participant

import numpy as np
import scipy.io
import os

trial_duration = 1.5  # Duration of each trial in seconds
segment_duration = 20  # Desired segment duration in seconds
trials_per_segment = int(segment_duration / trial_duration)  # Number of trials per 20-second segment
sampling_rate = 256

# Specify the channels to use
selected_channels = [3, 5, 8, 14, 17, 19, 24, 25, 36, 39, 40, 45, 47, 52, 53]
selected_channels = [ch - 1 for ch in selected_channels]  # Convert to 0-based indexing

# Loop through each test participant
for participant in test_participants:
    # Load the .mat file for the participant
    test_file_path = f'/content/drive/MyDrive/Dataset/splitset/{int(participant)}_test.mat'
    data = scipy.io.loadmat(test_file_path)

    # Extract the EEG data (assuming the key is 'test_data' in the .mat file)
    test_data = data['test_data']  # Shape: (timepoints, channels, trials)
    num_timepoints, num_channels, num_trials = test_data.shape

    # Create a directory to save the 20-second segments
    save_dir = f"/content/drive/MyDrive/Dataset/test_segments/participant_{participant}/"
    os.makedirs(save_dir, exist_ok=True)

    # Group trials into 20-second segments
    segment_idx = 1  # Initialize segment index
    for start in range(0, num_trials - trials_per_segment + 1, trials_per_segment):
        # Extract 20-second segment for selected channels
        segment_data = test_data[:, selected_channels, start:start + trials_per_segment]  # Shape: (timepoints, selected_channels, trials_per_segment)
        segment_signal = segment_data.reshape(-1, segment_data.shape[1])  # Flatten trials into a single time series
        segment_signal = segment_signal[:20 * sampling_rate, :]  # Ensure the segment is exactly 20 seconds

        # Save the 20-second segment
        np.save(f"{save_dir}/segment_{segment_idx}.npy", segment_signal)
        segment_idx += 1  # Increment segment index

    print(f"All 20-second segments saved for participant {participant} in {save_dir}")

All 20-second segments saved for participant 4 in /content/drive/MyDrive/Dataset/test_segments/participant_4/
All 20-second segments saved for participant 5 in /content/drive/MyDrive/Dataset/test_segments/participant_5/
All 20-second segments saved for participant 7 in /content/drive/MyDrive/Dataset/test_segments/participant_7/
All 20-second segments saved for participant 9 in /content/drive/MyDrive/Dataset/test_segments/participant_9/
All 20-second segments saved for participant 13 in /content/drive/MyDrive/Dataset/test_segments/participant_13/
All 20-second segments saved for participant 20 in /content/drive/MyDrive/Dataset/test_segments/participant_20/
All 20-second segments saved for participant 25 in /content/drive/MyDrive/Dataset/test_segments/participant_25/
All 20-second segments saved for participant 26 in /content/drive/MyDrive/Dataset/test_segments/participant_26/
All 20-second segments saved for participant 27 in /content/drive/MyDrive/Dataset/test_segments/participant_27/


In [4]:
import numpy as np
import os

# Loop through each test participant
for participant in test_participants:
    # Path where 20s segments are saved
    segment_dir = f"/content/drive/MyDrive/Dataset/test_segments/participant_{participant}/"
    segment_files = sorted([f for f in os.listdir(segment_dir) if f.endswith('.npy')])

    # Create a directory to save 60s segments
    save_dir = f"/content/drive/MyDrive/Dataset/test_segments_60s/participant_{participant}/"
    os.makedirs(save_dir, exist_ok=True)

    segment_idx = 1  # Initialize segment index

    # Combine every 3 consecutive 20s segments
    for i in range(0, len(segment_files) - 3 + 1, 3):
        # Load 3 segments
        seg1 = np.load(os.path.join(segment_dir, segment_files[i]))
        seg2 = np.load(os.path.join(segment_dir, segment_files[i+1]))
        seg3 = np.load(os.path.join(segment_dir, segment_files[i+2]))

        # Concatenate to form 60s segment
        segment_60s = np.concatenate([seg1, seg2, seg3], axis=0)  # Along time axis

        # Save the 60s segment
        np.save(f"{save_dir}/segment_{segment_idx}.npy", segment_60s)
        segment_idx += 1  # Increment segment index

    print(f"All 60-second segments saved for participant {participant} in {save_dir}")


All 60-second segments saved for participant 4 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_4/
All 60-second segments saved for participant 5 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_5/
All 60-second segments saved for participant 7 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_7/
All 60-second segments saved for participant 9 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_9/
All 60-second segments saved for participant 13 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_13/
All 60-second segments saved for participant 20 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_20/
All 60-second segments saved for participant 25 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_25/
All 60-second segments saved for participant 26 in /content/drive/MyDrive/Dataset/test_segments_60s/participant_26/
All 60-second segments saved for participant 27 in /content/drive/MyDrive/Datase